right now, this compares 9 channel CNN vs. GLMs, in terms of raw performance (cc^2 or cc), on ALL neurons (so no subset stuff yet)

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
from collections import OrderedDict
from itertools import product
import h5py
import numpy as np
import pandas as pd

In [3]:
from tang_jcompneuro.model_fitting_postprocess import load_model_performance

In [4]:
def check_all():
    subsets_to_check = ('all', 'OT')
    datasets_to_check = ('MkA_Shape', 'MkE2_Shape')
    
    models_to_examine = [
        ('cnn', 'b.9'),
        ('glm', 'linear_poisson'),
        ('glm', 'fpower_poisson'),
        ('glm', 'gqm.2_poisson'),
        ('glm', 'gqm.4_poisson'),
        ('glm', 'gqm.8_poisson'),
        ('glm', 'linear_gaussian'),
        ('glm', 'fpower_gaussian'),
        ('glm', 'gqm.2_gaussian'),
        ('glm', 'gqm.4_gaussian'),
        ('glm', 'gqm.8_gaussian'),
    ] # TODO should make this a ordered dict.
      # so that it will be easier to collect best val model for each model class.
    
    score_all = []
    for dataset, subset in product(datasets_to_check, subsets_to_check):
        
        print(dataset, subset)
        for (model_type, model_subtype) in models_to_examine: 
            score_new_cc = np.asarray(
                [load_model_performance(dataset, subset, 100, s, model_type, model_subtype)['corr'] for s in range(2)]
            ).mean(axis=0)
            
            score_new_cc2 = np.asarray(
                [load_model_performance(dataset, subset, 100, s, model_type, model_subtype)['corr']**2 for s in range(2)]
            ).mean(axis=0)
            
            score_all.append({
                'dataset': dataset,
                'subset': subset,
                'model': model_type + '_' + model_subtype,
                'cc': score_new_cc.mean(),
                'cc2': score_new_cc2.mean(),
                # later on, I can add neuron subset, etc.
            })
            
    score_all = pd.DataFrame(score_all,
                             columns = ['dataset', 'subset', 'model', 'cc', 'cc2']).set_index(['dataset', 'subset', 'model'], verify_integrity=True)
    return score_all

In [5]:
df_all = check_all()

MkA_Shape all
MkA_Shape OT
MkE2_Shape all
MkE2_Shape OT


In [6]:
df_all.columns.name = 'metric'

In [7]:
# this is the way to change a multiindex's names.
# df_all.index.rename(['haha','haha2','haha3'], inplace=True)

In [8]:
df_all.unstack('subset')

metric                                cc                 cc2          
subset                                OT       all        OT       all
dataset    model                                                      
MkA_Shape  cnn_b.9              0.337550  0.432107  0.160472  0.220729
           glm_fpower_gaussian  0.260039  0.289380  0.101413  0.101843
           glm_fpower_poisson   0.261663  0.292536  0.103336  0.104724
           glm_gqm.2_gaussian   0.221168  0.322815  0.078149  0.126313
           glm_gqm.2_poisson    0.231374  0.338436  0.086884  0.140991
           glm_gqm.4_gaussian   0.235860  0.337893  0.088910  0.138871
           glm_gqm.4_poisson    0.244452  0.352186  0.096849  0.152866
           glm_gqm.8_gaussian   0.234528  0.333564  0.089052  0.136867
           glm_gqm.8_poisson    0.237862  0.346025  0.093232  0.148912
           glm_linear_gaussian  0.193430  0.225797  0.059773  0.065544
           glm_linear_poisson   0.201861  0.236187  0.066026  0.073063
MkE2_Shape cnn_b.9              0.424945  0.534128  0.244821  0.323584
           glm_fpower_gaussian  0.312055  0.348596  0.143512  0.147058
           glm_fpower_poisson   0.309706  0.351439  0.141858  0.149461
           glm_gqm.2_gaussian   0.295394  0.424180  0.135937  0.212676
           glm_gqm.2_poisson    0.307597  0.445536  0.146406  0.235039
           glm_gqm.4_gaussian   0.302531  0.437969  0.141875  0.225771
           glm_gqm.4_poisson    0.309054  0.457312  0.146710  0.246269
           glm_gqm.8_gaussian   0.292490  0.429254  0.133827  0.217994
           glm_gqm.8_poisson    0.295629  0.446494  0.136623  0.235992
           glm_linear_gaussian  0.248554  0.300218  0.099965  0.111676
           glm_linear_poisson   0.262213  0.320696  0.112176  0.128884